# 🆕 LangChain Memory with LCEL (Modern Approach)

This notebook demonstrates memory management using **LCEL (LangChain Expression Language)**.

## What is LCEL?

LCEL is LangChain's **new standard approach** (2024~)

### Classic Chains vs LCEL

| Feature | Classic Chains | LCEL |
|---------|---------------|------|
| Status | ❌ Deprecated | ✅ Current Standard |
| Explicitness | ❌ Implicit ("magic") | ✅ Explicit |
| Streaming | ❌ Limited | ✅ Auto-supported |
| Async | ❌ Manual | ✅ Auto-supported |
| Batch | ❌ Limited | ✅ Auto-supported |
| Type Safety | ❌ Weak | ✅ Strong |
| Debugging | ❌ Difficult | ✅ Easy |

### LCEL Core: Pipe Operator (`|`)

```python
# Classic
chain = LLMChain(llm=llm, prompt=prompt)

# LCEL - More intuitive!
chain = prompt | llm
```

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

## 1. Basic LCEL Chain (without memory)

Let's start by creating a simple LCEL chain.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Define prompt
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Answer this question: {question}"
)

# LLM
llm = ChatOpenAI(temperature=0.0)

# Output parser
output_parser = StrOutputParser()

# LCEL chain: connect with pipe (|) operator!
chain = prompt | llm | output_parser

print("Chain created!")
print(f"Type: {type(chain)}")

In [ ]:
# Execute the chain
response = chain.invoke({"question": "What is 1+1?"})
print(response)

## 2. LCEL with Message History (adding memory)

Now let's add conversation history!

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [ ]:
# Dictionary to store message history per session
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Returns message history for the given session ID."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

print("Session history function created!")

In [ ]:
# Prompt with message history
prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Have a conversation with the human."),
    MessagesPlaceholder(variable_name="history"),  # 👈 Conversation history goes here
    ("human", "{input}")
])

print("Prompt with history created!")
print("\nPrompt structure:")
print(prompt_with_history)

In [ ]:
# Create LCEL chain
chain = prompt_with_history | llm

# Wrap with Runnable that automatically manages message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

print("✅ Chain with message history created!")

### Conversation Test - Remembering Name

In [ ]:
# First message - introduce name
response = chain_with_history.invoke(
    {"input": "Hi, my name is Andrew"},
    config={"configurable": {"session_id": "user123"}}
)
print("AI:", response.content)

In [ ]:
# Second message - check if name is remembered
response = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "user123"}}  # 👈 Same session_id!
)
print("AI:", response.content)

In [ ]:
# Third message
response = chain_with_history.invoke(
    {"input": "What is 1+1?"},
    config={"configurable": {"session_id": "user123"}}
)
print("AI:", response.content)

In [ ]:
# Fourth message - ask name again
response = chain_with_history.invoke(
    {"input": "What was my name again?"},
    config={"configurable": {"session_id": "user123"}}
)
print("AI:", response.content)

### View Saved Conversation History

In [ ]:
# View saved message history
session_history = store["user123"]
print("=== Full Conversation History ===")
print(f"Total messages: {len(session_history.messages)}\n")

for i, message in enumerate(session_history.messages, 1):
    role = "👤 Human" if message.type == "human" else "🤖 AI"
    print(f"{i}. {role}: {message.content}")
    print()

### New Session - Won't Remember Name

In [ ]:
# Use different session_id
response = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "user456"}}  # 👈 Different session_id!
)
print("AI (new session):", response.content)
print("\n✅ Different sessions don't share conversation history!")

## 3. 🚀 Powerful LCEL Features

### Streaming

In [ ]:
print("AI (streaming): ", end="", flush=True)

for chunk in chain_with_history.stream(
    {"input": "Tell me a short joke"},
    config={"configurable": {"session_id": "user123"}}
):
    print(chunk.content, end="", flush=True)

print("\n\n✅ Streaming allows receiving responses in real-time!")

### Batch Processing

In [ ]:
# Process multiple questions at once
questions = [
    {"input": "What is 2+2?"},
    {"input": "What is 3+3?"},
    {"input": "What is 4+4?"},
]

responses = chain_with_history.batch(
    questions,
    config={"configurable": {"session_id": "batch_test"}}
)

print("=== Batch Processing Results ===")
for q, r in zip(questions, responses):
    print(f"Q: {q['input']}")
    print(f"A: {r.content}")
    print()

### Async Processing

LCEL automatically supports async!

In [ ]:
import asyncio

async def async_example():
    response = await chain_with_history.ainvoke(
        {"input": "Hello from async!"},
        config={"configurable": {"session_id": "async_test"}}
    )
    return response.content

# Run in Jupyter
result = await async_example()
print("Async result:", result)

## 4. Window Memory (Remember Only Recent N Messages)

Implementing Classic's `ConversationBufferWindowMemory` with LCEL

In [ ]:
from typing import List
from langchain_core.messages import BaseMessage

class WindowedChatHistory(ChatMessageHistory):
    """History that keeps only the most recent N messages"""
    
    def __init__(self, window_size: int = 4):
        super().__init__()
        self.window_size = window_size
    
    @property
    def messages(self) -> List[BaseMessage]:
        """Returns only the most recent window_size messages"""
        all_messages = super().messages
        return all_messages[-self.window_size:] if len(all_messages) > self.window_size else all_messages

# Window history store
window_store = {}

def get_windowed_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in window_store:
        window_store[session_id] = WindowedChatHistory(window_size=2)  # Keep only 2 recent
    return window_store[session_id]

# Chain using window memory
chain_with_window = RunnableWithMessageHistory(
    chain,
    get_windowed_history,
    input_messages_key="input",
    history_messages_key="history",
)

print("✅ Window memory chain created (k=2)")

In [ ]:
# Send multiple messages
session = "window_test"

messages = [
    "My name is Alice",
    "I like pizza",
    "I live in Seoul",
    "What do you remember about me?"
]

for msg in messages:
    response = chain_with_window.invoke(
        {"input": msg},
        config={"configurable": {"session_id": session}}
    )
    print(f"User: {msg}")
    print(f"AI: {response.content}")
    print()

print("\n💡 With window size of 2, the first message (name) should be forgotten!")

## 5. 📊 Classic vs LCEL Comparison

### Classic Approach (Deprecated)

```python
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory

# ❌ Implicit behavior
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# ❌ No session management
# ❌ Limited streaming
# ❌ Difficult batch processing
response = conversation.predict(input="Hi, my name is Andrew")
```

### LCEL Approach (Current Standard)

```python
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# ✅ Explicit prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# ✅ Clear chain composition
chain = prompt | llm

# ✅ Session-based history management
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# ✅ Supports invoke, stream, batch, ainvoke
response = chain_with_history.invoke(
    {"input": "Hi, my name is Andrew"},
    config={"configurable": {"session_id": "abc123"}}
)
```

## 🎯 Why Use LCEL?

### 1. Explicitness
- Code clearly shows what's happening
- No hidden "magic" behavior

### 2. Composability
```python
# Easy to compose chains
chain1 = prompt1 | llm
chain2 = prompt2 | llm
combined = chain1 | transform | chain2
```

### 3. Automatic Features
- ✅ Streaming
- ✅ Batch
- ✅ Async
- ✅ Parallel execution
- ✅ Fallbacks
- ✅ Retries

### 4. Debugging
```python
# Perfect integration with LangSmith
# Clear tracking of each step
```

### 5. Type Safety
```python
# IDE autocomplete
# Type checking
# Fewer runtime errors
```

## 📚 Summary

### Classic Chains (L2-Memory.ipynb)
- ❌ **Deprecated** (since 2024)
- ❌ Implicit behavior
- ❌ Limited features
- ✅ Simple to use (beginner-friendly)

### LCEL (this notebook)
- ✅ **Current Standard** (2024~)
- ✅ Explicit and transparent code
- ✅ Intuitive with pipe operator (`|`)
- ✅ Auto-support for streaming, batch, async
- ✅ Type safety
- ✅ Better debugging
- ✅ Complex chain composition
- ✅ Session-based memory management

### Recommendations
1. **New projects**: Always use LCEL
2. **Existing projects**: Migrate to LCEL when possible
3. **Learning**: Understand both, but prioritize LCEL

### Next Steps
- L3-chains.ipynb: Build complex chains with LCEL
- L4-QnA.ipynb: Implement RAG with LCEL
- [LangChain LCEL Documentation](https://python.langchain.com/docs/expression_language/)